# NeuroHub Tools and Tasks

## 0. Set up boilerplate

Make sure the optional package is installed then import the packages we'll use.

In [ ]:
!pip install requests

In [ ]:
import requests
import json

All API requests are done with respect to the `base_url` and with the API token from [NeuroHub New API Token](https://portal.neurohub.ca/nh_users/new_token).

In [ ]:
base_url = 'https://portal.cbrain.mcgill.ca'
cbrain_api_token = "PASTE_NEUROHUB_API_TOKEN_HERE"

## 1. Session

Do we have a valid session?

In [ ]:
session_response = requests.post(
    url = '/'.join([base_url, 'session']),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)

if session_response.status_code != requests.codes.ok:
    print('Session is not valid.')
    print(json.dumps(session_response.json(), indent=4))
else:
    print('Session is valid.')
    session_info = session_response.json()
    user_id = str(session_info['user_id'])
    print("User ID:", user_id)

## 2. Tool Configs / Pipelines

List all possible pipelines.  That is, list all tool configurations where a particular tool can run on a particular bourreau.

In [ ]:
tool_configs = []
tool_configs_request = {
    'cbrain_api_token': cbrain_api_token,
    'page': 1,
    'per_page': 1000
}

while True:

    tool_configs_response = requests.get(
        url = '/'.join([base_url, 'tool_configs']),
        data = tool_configs_request,
        headers = {'Accept': 'application/json'}
    )

    if tool_configs_response.status_code != requests.codes.ok:
        print('Tools request failed.')
        print(tool_configs_response)
        break

    tool_configs += tool_configs_response.json()
    tool_configs_request['page'] += 1
    
    if len(tool_configs_response.json()) < tool_configs_request['per_page']:
        break 
    
print("{} tool configs found\n".format(str(len(tools))))
print(json.dumps(tool_configs, indent=4))

As we did before with user files, we can now filter the tools by words in their description. Let's filter to find all tools with the word `FSL` in the description. 

The tool config ID is the `id` field. You can use the `bourreau_id` and `tool_id` to get information about these and verify this is the correct pipeline you wish to use.

In [ ]:
result = list(filter(lambda f: 'FSL' in f['description'], tools))
print(json.dumps(result, indent=4))

## 3. Submit a task

TODO

For this example we will use submit a task using a particular tool config (`id: 721`) with a `.nii.gz` file to FSLFirst (`tool_id: 51`) on Converter-2 (`bourreau_id: 39`).

**Note that this needs a valid userfile ID to run. A public data provider to hold example files for this notebook is in the works.**

In [ ]:
#Store the ID of the file we wish to run a task on. In this case, the first filtered result from the
#files loaded in the previous section. Feel free to change it to a brainscan you have access to.
fileID = result[0]['id']

#Write the post API call
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}
params = (
    ('cbrain_api_token', cbrain_api_token),
)

#Specify the parameters of the task in a dictionary. We are running FSLFirst to register the brain scan.
data = {
  "cbrain_task": {
    'type': 'CbrainTask::FslFirst', 
    'user_id': user_id,
    'tool_config_id': 721, #This specifies the Tool we are running, and the server we are running it on.
    'params': { #These parameters vary from tool to tool.
      'interface_userfile_ids': [fileID], 
      'input_file': fileID, 
      'prefix': 'output', 
      'brain_extracted': False, 
      'three_stage': False, 
      'verbose': False       
    }, 
    'run_number': None, 
    'results_data_provider_id': 179, 
    'cluster_workdir_size': None, 
    'workdir_archived': True, 
    'description': ''}
}

#Convert our parameter dictionary to a json and post the task.
y = json.dumps(data)
response = requests.post('https://portal.cbrain.mcgill.ca/tasks', headers=headers, params=params, data=y)

if response.status_code == 200:
    print(response.text)
    json_format = json.loads(response.text) #save the response to a json file for future use
else:
    print("Task posting failed.")
    print(response.text)

Let's track the completion status of the task.

In [ ]:
taskID = json_format[0]["id"]
headers = {
    'Accept': 'application/json',
}
params = (
    ('id', taskID),
    ('cbrain_api_token', cbrain_api_token)
)
url = 'https://portal.cbrain.mcgill.ca/tasks/' + str(taskID)
response = requests.get(url, headers=headers, params=params)
if response.status_code == 200:
    print(response.text)
else:
    print("Task Info retrieval failed.")